# Module 6 Assignment

A few things you should keep in mind when working on assignments:

1. Make sure you fill in any place that says `# YOUR CODE HERE`. Do not write your answer anywhere else other than where it says `# YOUR CODE HERE`. Anything you write elsewhere will be removed or overwritten by the autograder.
2. Before you submit your assignment, make sure everything runs as expected. Go to the menubar, select Kernel, and restart the kernel and run all cells (Restart & Run all).
3. Do not change the title (i.e. file name) of this notebook.
4. Make sure that you save your work (in the menubar, select File → Save and CheckPoint).
5. All work must be your own, if you do use any code from another source (such as a course notebook or a website) you need to properly cite the source.

-----

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import sklearn.preprocessing
from sklearn.model_selection import train_test_split

import helper as hp
from nose.tools import assert_equal, assert_almost_equal, assert_true, assert_is_instance

-----

## Loading Breast Cancer Data

In this assignment, we will work with a breast cancer data set to make predictive models. Before we build a model, we first load the data into the assignment notebook, and randomly sample several rows. The second Code cell creates the features and labels.

-----

In [2]:
# Load data
df = pd.read_csv('./breast-cancer-wisconsin.data')
df.sample(5)

,id,clump thickness,uniformity cell size,uniformity cell shape,marginal adhesion,epithelial cell size,bare nuclei,bland chromatin,normal nucleoli,mitoses,class
363,654244,1,1,1,1,1,1,2,1,1,2
361,636437,1,1,1,1,2,1,1,1,1,2
252,255644,10,5,8,10,3,10,5,1,3,4
537,1299924,3,2,2,2,2,1,4,2,1,2
355,428598,1,1,3,1,1,1,2,1,1,2


In [3]:
# Create features and labels
labels = df['class']
features = df.drop('class', axis=1)

-----

## Problem 1:  Variance Thresholding

In this problem, you will create and implement a function to perform variance thresholding. Specifically, you must complete the following tasks:

- Define a function called `VarianceThreshold` that excepts three arguments: `features`, `labels`, and `threshold`, in this order.
- Create a [`Variance Threshold`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html) feature selector by using the scikit learn library.
  - Use default parameters for this selector, except
  - Set the `threshold` parameter to the `threshold` argument provided to your function.
- Apply this feature selector to fit and transform the feature and label arguments provided to your function.
- Return the selected features and variances of the individual features, in this order.  

**TIP**: Notice that the function you write will be defined as `VarianceThreshold`. Thus, you should not import VarianceThreshold from sklearn.feature.selection. Instead, use the _dot_ operator to reference the `VarianceThreshold` class from the `feature_selection` module in the scikit learn library.

-----

In [4]:
from  sklearn import feature_selection

### YOUR CODE HERE

def VarianceThreshold(features, labels, threshold):
    
    # Create a Variance Threshold feature selector
    selector = feature_selection.VarianceThreshold(threshold=threshold)
    
    
    # Apply the feature selector to fit and transform the features and labels
    selected_features = selector.fit_transform(features, labels)
    
    # Get the variances of the individual features
    variances = selector.variances_
    
    return selected_features, variances

In [5]:
# Select features and compute relevant variances
selected_features, variences = VarianceThreshold(features, labels, 6)

# Test function
assert_equal(hp.vt_sf, selected_features.flatten().tolist())
assert_equal(variences.flatten().tolist(), [384635052873.4785, 7.945044792052974,
     9.381357331041032, 8.918538272070725, 8.193702316667704, 4.934873062387323,
     13.258254749844047, 5.992227040723361, 9.30512830956357, 2.9977641487794995])

-----

## Problem 2: Recursive Feature Extraction
In this problem, you will create and implement a function to perform recursive feature extraction. Specifically, you must complete the following tasks:

- Write a function called `RFE` that accepts four parameters: features, label, rs, and n, in this order.
- Create a random forest classifier and assign the `random_state` parameter for this estimator to the `rs` argument passed into your function.
- Create a Recursive Feature Estimator (RFE) by using the scikit learn library's [RFE](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE) object. 
  - Use the random forest classifier as the estimator for the `RFE` object.
  - Assign the numbers of features to select parameter to `n`, which was passed as an argument into your function.
- Fit the RFE estimator by using the features and labels.
- Return the RFE model.

When completing this problem, you should reference the **TIP** provided with Problem 1: Variance Thresholding.

-----

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import feature_selection

### YOUR CODE HERE

def RFE(features, labels, rs, n):
    
    # Create classifier
    rfc = RandomForestClassifier(random_state=rs)
    
    # Perform RFE
    rfe = feature_selection.RFE(estimator=rfc, n_features_to_select=n)
    
    # Fit features plus noise
    fs = rfe.fit(features, labels)
        
    return fs

In [7]:
# Create RFE estimator
rfe1 = RFE(features, labels, rs=0, n=1)

# Test function results
assert_equal(rfe1.ranking_.tolist(), [9, 7, 2, 1, 8, 5, 4, 3, 6, 10])

for i in range(1, len(features.columns)+1):
    rfe = RFE(features, labels, rs=0, n=i)
    print ('Performing Recursive Feature Selection. Choosing', i, 'features.')
    if i == 1:
        assert_equal(rfe.ranking_.tolist(), [9, 7, 2, 1, 8, 5, 4, 3, 6, 10])
        assert_equal(rfe.support_.tolist(), [False, False, False, True, False, 
                                             False, False, False, False, False])
    elif i == 2:
        assert_equal(rfe.ranking_.tolist(), [8, 6, 1, 1, 7, 4, 3, 2, 5, 9])
        assert_equal(rfe.support_.tolist(), [False, False, True, True, False, 
                                             False, False, False, False, False])
    elif i == 3:
        assert_equal(rfe.ranking_.tolist(), [7, 5, 1, 1, 6, 3, 2, 1, 4, 8])
        assert_equal(rfe.support_.tolist(), [False, False, True, True, False, 
                                             False, False, True, False, False])
    elif i == 4:
        assert_equal(rfe.ranking_.tolist(), [6, 4, 1, 1, 5, 2, 1, 1, 3, 7])
        assert_equal(rfe.support_.tolist(), [False, False, True, True, False, 
                                             False, True, True, False, False])
    elif i == 5:
        assert_equal(rfe.ranking_.tolist(), [5, 3, 1, 1, 4, 1, 1, 1, 2, 6])
        assert_equal(rfe.support_.tolist(), [False, False, True, True, False, 
                                             True, True, True, False, False])
    elif i == 6:
        assert_equal(rfe.ranking_.tolist(), [4, 2, 1, 1, 3, 1, 1, 1, 1, 5])
        assert_equal(rfe.support_.tolist(), [False, False, True, True, False, 
                                             True, True, True, True, False])
    elif i == 7:
        assert_equal(rfe.ranking_.tolist(), [3, 1, 1, 1, 2, 1, 1, 1, 1, 4])
        assert_equal(rfe.support_.tolist(), [False, True, True, True, False, True, 
                                             True, True, True, False])
    elif i == 8:
        assert_equal(rfe.ranking_.tolist(), [2, 1, 1, 1, 1, 1, 1, 1, 1, 3])
        assert_equal(rfe.support_.tolist(), [False, True, True, True, True, True, 
                                             True, True, True, False])
    elif i == 9:
        assert_equal(rfe.ranking_.tolist(), [1, 1, 1, 1, 1, 1, 1, 1, 1, 2])
        assert_equal(rfe.support_.tolist(), [True, True, True, True, True, True, 
                                             True, True, True, False])
    else:
        assert_equal(rfe.ranking_.tolist(), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
        assert_equal(rfe.support_.tolist(), [True, True, True, True, True, True, 
                                             True, True, True, True])


Performing Recursive Feature Selection. Choosing 1 features.
Performing Recursive Feature Selection. Choosing 2 features.
Performing Recursive Feature Selection. Choosing 3 features.
Performing Recursive Feature Selection. Choosing 4 features.
Performing Recursive Feature Selection. Choosing 5 features.
Performing Recursive Feature Selection. Choosing 6 features.
Performing Recursive Feature Selection. Choosing 7 features.
Performing Recursive Feature Selection. Choosing 8 features.
Performing Recursive Feature Selection. Choosing 9 features.
Performing Recursive Feature Selection. Choosing 10 features.


-----

For the final problem below, you will use the car data to relate car price to the car specifications. In the following Code cells, we first load these data and sample several instances, before cleaning the data for non-numeric features, and finally, extracting features (`X`) and labels (`y`).

-----

In [8]:
# Load data
df = pd.read_csv('./imports-85.data')
df.sample(5)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,'engine-location',wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio',horsepower,peak-rpm,city-mpg,highway,price
9,0,122,1,1,turbo,2.0,3,4wd,front,99.5,...,131,mpfi,3.13,3.40,7.0,160,5500,16,22,13207.129353
195,-1,74,21,1,std,4.0,1,rwd,front,104.3,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,13415.000000
121,1,154,14,1,std,4.0,2,fwd,front,93.7,...,90,2bbl,2.97,3.23,9.4,68,5500,31,38,6692.000000
7,1,122,1,1,std,4.0,1,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,18920.000000
131,2,122,16,1,std,2.0,3,fwd,front,96.1,...,132,mpfi,3.46,3.90,8.7,?,?,23,31,9895.000000


In [9]:
# Eliminate non-numeric features
df_simple = df[df.columns[df.dtypes!=object]]
df_simple.head()

,symboling,normalized-losses,make,fuel-type,num-of-doors,body-style,wheel-base,length,width,height,curb-weight,engine-size,compression-ratio',city-mpg,highway,price
0,3,122,0,1,2.0,4,88.6,168.8,64.1,48.8,2548,130,9.0,21,27,13495.0
1,3,122,0,1,2.0,4,88.6,168.8,64.1,48.8,2548,130,9.0,21,27,16500.0
2,1,122,0,1,2.0,3,94.5,171.2,65.5,52.4,2823,152,9.0,19,26,16500.0
3,2,164,1,1,4.0,2,99.8,176.6,66.2,54.3,2337,109,10.0,24,30,13950.0
4,2,164,1,1,4.0,2,99.4,176.6,66.4,54.3,2824,136,8.0,18,22,17450.0


In [10]:
# Create feature and label arrays
X = df_simple.drop("price", axis=1)
y = np.ravel(df_simple["price"])

-----

## Problem 3: Principal Component Analysis

For this problem, you will complete the function `fit_pca`, provided below, to perform principal component analysis on a provided DataFrame, `df`. To complete this task, you must create an instance of the `PCA` estimator in the scikit learn library, specifying the number of components to keep (which is passed into the `fit_pca` function via the `n_c` parameter). Using this estimator, you should fit and transform the DataFrame to compute a NumPy array that contains the reduced set of features.

-----

In [11]:
from sklearn.decomposition import PCA

def fit_pca(df, n_c):
    """
    Perform principal component analysis
    
    Parameters
    ----------
    df: A pandas DataFrame containing the relevant features
    n_c: The number of principal components to keep
    
    Returns
    -------
    reduced: A NumPy array containing the PCA reduced features
    """
    
    ### YOUR CODE HERE
    
    pca = PCA(n_components=n_c)
    
    reduced = pca.fit_transform(df)
    
    
    return reduced

In [12]:
# Compute reduced features on car data
pca_data = fit_pca(X,n_c=X.shape[1])

# Test function
assert_almost_equal(sum(pca_data[0]), 0.60937, places=4)
assert_equal(pca_data.shape, (205, 15))

**&copy; 2017: Robert J. Brunner at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode 